In [1]:
import os
import numpy as np
import torch
import scipy.io

In [2]:
channels = 6
samples = 4096
rate = 1024
path = './Base de Datos Habla Imaginada'

data = []
labels = []

for root, _, files in os.walk(path):
    for file in files:
        if file.endswith('_EEG.mat'):
            mat = scipy.io.loadmat(os.path.join(root, file))
            # The format stores the per channel data + three labels.
            data.append(mat['EEG'][:, :channels*samples].reshape(-1, channels, samples))
            labels.append(mat['EEG'][:, channels*samples:])

data = torch.from_numpy(np.concatenate(data, axis=0)).to(torch.float32)
labels = torch.from_numpy(np.concatenate(labels, axis=0)).to(int)     

In [3]:
data.shape, labels.shape

(torch.Size([9315, 6, 4096]), torch.Size([9315, 3]))

In [4]:
data[0], labels[0]

(tensor([[-13.2110, -11.7757, -10.2983,  ...,  -6.5553,  -6.2022,  -5.6591],
         [ -2.7764,  -2.3330,  -1.8796,  ...,   2.8046,   2.3710,   1.7605],
         [  3.6162,   3.7574,   3.8462,  ...,   0.8512,   0.8198,   0.6374],
         [  6.7315,   6.5819,   6.4022,  ...,   7.1031,   6.9845,   6.7511],
         [  6.8673,   7.1598,   7.4232,  ...,  -2.0913,  -1.7007,  -1.3703],
         [  8.4799,   8.3454,   8.1901,  ...,   6.1206,   6.1089,   5.9817]]),
 tensor([1, 8, 2]))

In [5]:
metadata = {
    "channels": channels,
    "channels_residual": channels + 1, # Add residual
    "samples": samples,
    "rate": rate,
    "channel_names": [a + n for a in "FCP" for n in "34"] + ["residual"],
    "stimuli": ["A", "E", "I", "O", "U", "Arriba", "Abajo", "Adelante", "Atras", "Derecha", "Izquierda"],
    "modalities": ["Imagined", "Pronounced"],
    "artifacts": ["None", "Blink"]
}

dataset = {
    "data": data,
    "labels": labels - 1, # YOLO: make 0-based
    "metadata": metadata
}

torch.save(dataset, "dataset.pt")